In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings(action='ignore')


In [2]:
train_data = pd.read_csv('../input/paddy-disease-classification/train.csv')
sub_data = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv')

In [3]:
train_data['variety'].value_counts()

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['variety'] = le.fit_transform(train_data['variety'])

In [5]:
train_data.isnull().sum()

In [6]:
sub_data

In [7]:
diseases_types=sorted(list(set(train_data['label'])))
n_classes=len(diseases_types)
print(n_classes)
print(diseases_types)

In [8]:
train= tf.keras.utils.image_dataset_from_directory("/kaggle/input/paddy-disease-classification/train_images/",
                                                        labels='inferred',
                                                        label_mode='categorical',
                                                        class_names=diseases_types,
                                                        color_mode='rgb',
                                                        image_size=(680,480),
                                                        shuffle=True,
                                                        validation_split=0.2,
                                                        subset='training',
                                                        seed=42)

val= tf.keras.utils.image_dataset_from_directory("/kaggle/input/paddy-disease-classification/train_images/",
                                                        labels='inferred',
                                                        label_mode='categorical',
                                                        class_names=diseases_types,   
                                                        color_mode='rgb',
                                                        image_size=(680,480),
                                                        shuffle=True,
                                                        validation_split=0.2,
                                                        subset='validation',
                                                        seed=42)

In [9]:
train

In [10]:
val

In [11]:
from tensorflow.keras import models,layers

In [12]:
model = models.Sequential()
pretrained_model=tf.keras.applications.ResNet50(include_top=False,input_shape=(680,480,3),pooling='avg',classes=10,weights='imagenet')
    
for layer in pretrained_model.layers:
    layer.trainable=False
    
    
model.add(pretrained_model)
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(99,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [13]:
model.summary()

In [14]:
tf.keras.utils.plot_model(model)

In [15]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train,validation_data=val,epochs=15)

In [16]:
test_data=tf.keras.utils.image_dataset_from_directory("/kaggle/input/paddy-disease-classification/test_images/",
                                                        labels=None,
                                                        label_mode=None,
                                                        color_mode='rgb',
                                                        image_size=(680,480),
                                                        shuffle=False)

In [17]:
predict=tf.argmax(model.predict(test_data),axis=-1)
predict

In [18]:
predictions=[]
for i in predict:
    predictions.append(diseases_types[i])

In [19]:
sub_data=sub_data.drop(columns='label',axis=1)
sub_data

In [21]:
submission=pd.DataFrame({'image_id':sub_data['image_id'],'label':predictions})
submission

In [23]:
submission.to_csv('paddy.csv',index=None)